# SS3 Diffusion

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

# Reproducibility
rng = np.random.default_rng(42)

# Plot defaults
plt.rcParams.update({
    "figure.figsize": (7, 4),
    "axes.grid": False,
    "axes.spines.top": False,
    "axes.spines.right": False,
})

**ADD CODE WHERE YOU SEE `YOUR CODE HERE` IN THE SNIPPETS**


## Section 1: SI and Bass

### Simple Contagion: SI Model

The **SI model** is the simplest epidemic-like model of diffusion.

We divide the population into:
- $s(t)$: share of susceptibles
- $i(t)$: share of infected/adopters


\begin{align*}
\dot{s}(t) &= - \beta\, s(t)\, i(t) \\
\dot{i}(t) &= + \beta\, s(t)\, i(t)
\end{align*}



**Question:**, what model is this equivalent to knowing that \(s+i=1\), 

**Answer:**





In [2]:
def si_dt(y, t, beta):
    s, i = y

    # YOUR CODE HERE
    dsdt = -beta * s * i
    didt =  beta * s * i
    return [dsdt, didt]

In [ ]:
# Plot S and I
from ipywidgets import interact

def plot_si(beta, i0):
    t = np.linspace(0, 10, 400)
    s0   = 1.0 - i0
    y0   = [s0, i0]
    
    sol = odeint(si_dt, y0, t, args=(beta,))
    s, i = sol.T

    incidence = beta * s * i
    
    plt.figure()
    #plt.plot(t, s, label="s(t) susceptible")
    plt.plot(t, i, label="i(t) infected")
    plt.plot(t, incidence, label="incidence ẋ = β s i")
    plt.xlabel("time")
    plt.ylabel("share")
    plt.title("SI model (no recovery)")
    plt.legend()

interact(plot_si, beta=(0, 1., .1), i0=(0, 1, .1))

interactive(children=(FloatSlider(value=0.5, description='beta', max=1.0), FloatSlider(value=0.0, description=…

<function __main__.plot_si(beta, i0)>

**Exercise:**

Change parameters $\beta$ and $i_0$. How do (a) speed and (b) peak incidence change?

# Bass Diffusion Model

The **Bass model** extends simple contagion to model technology adoption.

State variable:
- $x(t)$: cumulative share of adopters, $0 \leq x(t) \leq 1$.

Hazard (adoption intensity) for a non-adopter:
$$
g(x) = p \;+\; q\,x
$$

Dynamics:
$$
\dot{x}(t) = \bigl(p + q\,x(t)\bigr)\,(1 - x(t)).
$$

**Question:** The bass model with $p=0$ or $q=0$ correspond to which models we discussed in the lectures? 

**Answer:**
- $q = 0 \Rightarrow$ pure external influence (“common source”).
- $p = 0 \Rightarrow$ pure imitation (like SI with a seed).


In [4]:
def bass_dt(x, t, p, q):
    # YOUR CODE HERE
    dxdt = (p + q * x) * (1-x)
    return dxdt

In [5]:
def plot_bass(p, q, x0):
    t = np.linspace(0, 20, 600)


    # YOUR CODE HERE (look at examble above)
    x = odeint(bass_dt, x0, t, args=(p, q))

    non_adopters = 1 - x
    incidence = bass_dt(x, t, p, q)  # dx/dt

    plt.figure()
    plt.plot(t, x, label="x(t): cumulative adopters")
    plt.plot(t, non_adopters, label="1 - x(t): non-adopters")
    plt.plot(t, incidence, label="dx/dt: incidence (sales)")
    plt.xlabel("time")
    plt.ylabel("share - dx/dt")
    plt.title("Bass model")
    plt.legend()


interact(plot_bass, p=(0, 1, .1), q=(0, 1, .1), x0=(0, 1, .1))

interactive(children=(FloatSlider(value=0.0, description='p', max=1.0), FloatSlider(value=0.0, description='q'…

<function __main__.plot_bass(p, q, x0)>

**Exercise:**

1) Hold $p$ fixed, increase $q$. What happens to S-shape and peak incidence?
2) Hold $q$ fixed, vary $p$. What changes early growth and timing?

## Section 2: SIR Models

The **SIR model** introduces recovery, so epidemics eventually end.

Compartments:
- $s(t)$: susceptible
- $i(t)$: infected/adopters
- $r(t)$: recovered/removed

$$
\begin{aligned}
\dot{s}(t) &= -\beta\, s(t)\, i(t) \\
\dot{i}(t) &= \beta\, s(t)\, i(t) - \gamma\, i(t) \\
\dot{r}(t) &= \gamma\, i(t)
\end{aligned}
$$

The **basic reproduction number** is
$$
\mathcal R_0 = \frac{\beta}{\gamma}.
$$


**Question:** When does an epidemic take off, when does it start? hint $\mathcal R_0$ and $\mathcal R_t$  

**Answer:**

In [6]:
# %% SIR MODEL (Recovery -> epidemic turns over)

def sir_dt(y, t, beta, gamma):
    s, i, r = y

    dsdt = # YOUR CODE HERE
    didt = # YOUR CODE HERE
    drdt = # YOUR CODE HERE
    return [dsdt, didt, drdt]


SyntaxError: invalid syntax (1561220853.py, line 6)

In [ ]:
## FILL IN THE MISSING CODE

def plot_sir(beta=.1, gamma=.2, i0=.1, r0=.0):
    
    s0 = 1 - i0 - r0
    y0 = [s0, i0, r0]
    
    
    t = np.linspace(0, 40, 800)
    
    sol = # YOUR CODE HERE
    
    s, i, r = sol.T
    
    # Figure Plotting
    plt.figure()
    plt.plot(t, s, label="s(t)", color='green')
    plt.plot(t, i, label="i(t)")
    plt.plot(t, r, label="r(t)")
    plt.xlabel("time")
    plt.ylabel("share")
    plt.title("SIR dynamics")
    
    

    R0 = beta / gamma
    
    if R0 > 1:
        plt.axhline(1/R0, linestyle="--", label="s = 1/R0 (threshold)", color='green')
    
    plt.legend();

interact(plot_sir);

**Exercise**
1) Change β and γ keeping R0 constant. What changes? What stays similar?
2) For R0<1 (e.g., β=0.2, γ=0.3), what happens to i(t)?

## Section 3: Complex Contagion on Networks

So far we assumed a *well-mixed* population.  
But many behaviors require **multiple exposures** from different peers.

We use a **threshold model**:

- Graph $ \mathcal G = (V,E) $, nodes $v \in V$.
- Each node state $x_v(t) \in \{0,1\}$.
- $\mathcal N(v)$ are the neighbors of $v$
- $x_v(t)$ is the state of node $v$ at time $t$: 0=non-adopter; 1=adopter

Threshold rule:

$$
x_v(t{+}1) =
\begin{cases}
1, & \sum_{u \in \mathcal N(v)} x_u(t) \;\ge\; \theta \\
x_v(t), & \text{otherwise.}
\end{cases}
$$

That is: a node adopts if at least $\theta$ of its neighbors have adopted.

In [ ]:
import networkx as nx
from ipywidgets import interact, Dropdown, IntSlider


# default seeds
DEFAULT_SEEDS = [1, 5, 4]
RNG_SEED = 42
np.random.seed(RNG_SEED)

graphs = {
    "Complete K8"              : nx.complete_graph(8),
    "Star (center + 10 leaves)": nx.star_graph(10),  # nodes 0..10, hub = 0
    "Chain (path, n=30)"       : nx.path_graph(30),
    "Ring lattice (n=20, k=6)" : nx.watts_strogatz_graph(20, 6, 0, seed=RNG_SEED),
    "Erdős–Rényi (n=30, p=0.05)": nx.gnp_random_graph(30, 0.05, seed=RNG_SEED),
    "Erdős–Rényi (n=30, p=0.10)": nx.gnp_random_graph(30, 0.10, seed=RNG_SEED),
    "Erdős–Rényi (n=30, p=0.20)": nx.gnp_random_graph(30, 0.20, seed=RNG_SEED),
}


plt.rcParams.update({
    "figure.figsize": (4.5, 4.5),
    "axes.spines.top": False,
    "axes.spines.right": False,
})


In [ ]:
def contagion_step(G, infected, threshold):
    """
    One synchronous update:
      - 'infected' is a list[bool] indexed by node id (0..N-1).
      - Mutates 'infected' in place; returns number of newly infected nodes.
    """
    to_flip = []
    for v in G.nodes():
        if not infected[v]:
            k = sum(infected[u] for u in G.neighbors(v))

            # YOUR CODE HERE (infected neighbors >= threshold)
            if #YOUR CODE HERE
                to_flip.append(v)
                
    for v in to_flip: # apply simultaneously
        infected[v] = True
    return len(to_flip)


In [ ]:
def simulate_with_history(G, theta, seeds, max_steps=10_000):
    """
    Returns: list of infection states from t=0 to fixed point.
    Each state is a list[bool] with length = number of nodes in G.
    """
    N = G.number_of_nodes()
    infected = [False] * N
    seeds = [s for s in seeds if s in G.nodes()]
    if not seeds:
        seeds = [0]
    for s in seeds:
        infected[s] = True

    history = [infected.copy()]
    for _ in range(max_steps):
        changed = contagion_step(G, infected, theta)
        if changed == 0:
            break
        history.append(infected.copy())
    return history


In [ ]:

def draw_network(G, infected, pos=None, title=""):
    if pos is None:
        pos = nx.spring_layout(G, seed=RNG_SEED)
    fig, ax = plt.subplots()
    adopted = [n for n, b in enumerate(infected) if b]
    others  = [n for n, b in enumerate(infected) if not b]
    nx.draw_networkx_nodes(G, pos, nodelist=others,  node_color="lightgrey",
                           edgecolors="black", ax=ax)
    nx.draw_networkx_nodes(G, pos, nodelist=adopted, edgecolors="black", ax=ax)
    nx.draw_networkx_edges(G, pos, alpha=0.5, ax=ax)
    nx.draw_networkx_labels(G, pos, font_size=8, ax=ax)
    ax.set_axis_off()
    ax.set_title(title)
    plt.show()


In [ ]:
# %% Minimal interaction: dropdown for graph, sliders for θ and step
def _visualize(graph_name, theta, step):
    G   = graphs[graph_name]
    pos = nx.spring_layout(G, seed=RNG_SEED)   # stable layout per graph
    seeds = [s for s in DEFAULT_SEEDS if s in G.nodes()]
    history = simulate_with_history(G, theta=theta, seeds=seeds)

    # Clamp step to the computed horizon
    s = min(step, len(history) - 1)

    # Fraction adopted over time
    N = G.number_of_nodes()
    frac = [sum(h)/N for h in history]

    # --- Two-panel figure ---
    fig, (ax_net, ax_frac) = plt.subplots(1, 2, figsize=(10, 4.5))

    # Left: network
    adopted = [n for n, b in enumerate(history[s]) if b]
    others  = [n for n, b in enumerate(history[s]) if not b]
    nx.draw_networkx_nodes(G, pos, nodelist=others, node_color="lightgrey",
                           edgecolors="black", ax=ax_net)
    nx.draw_networkx_nodes(G, pos, nodelist=adopted, edgecolors="black", ax=ax_net)
    nx.draw_networkx_edges(G, pos, alpha=0.5, ax=ax_net)
    nx.draw_networkx_labels(G, pos, font_size=8, ax=ax_net)
    ax_net.set_axis_off()
    ax_net.set_title(f"{graph_name}\nθ={theta}, seeds={seeds}\nstep {s}/{len(history)-1}")

    # Right: fraction adopted
    ax_frac.plot(range(len(frac)), frac, marker="o", color="black")
    ax_frac.plot(s, frac[s], "ro", markersize=10)  # red dot at current step
    ax_frac.set_ylim(0, 1.05)
    ax_frac.set_xlabel("step")
    ax_frac.set_ylabel("fraction adopted")
    ax_frac.set_title("Adoption over time")

    plt.tight_layout()
    plt.show()

# Fixed max=30 keeps the widget simple; we clamp if history is shorter.
interact(
    _visualize,
    graph_name=Dropdown(options=list(graphs.keys()), value="Star (center + 10 leaves)", description="Graph"),
    theta=IntSlider(value=2, min=1, max=6, step=1, description="θ"),
    step=IntSlider(value=0, min=0, max=30, step=1, description="step"),
);
